# 04 - Model Training

This notebook trains and tunes prediction models.

## Objectives:
- Split data into train/test sets
- Train multiple models
- Hyperparameter tuning
- Cross-validation
- Model comparison

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import joblib

# Load engineered features
df = pd.read_csv('../data/processed/movies_engineered.csv')
print(f"Data shape: {df.shape}")

In [ ]:
# Prepare features and target
# X = df.drop('target_column', axis=1)
# y = df['target_column']
# print(f"Features shape: {X.shape}")
# print(f"Target shape: {y.shape}")

In [ ]:
# Train/test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(f"Training set size: {X_train.shape}")
# print(f"Test set size: {X_test.shape}")

In [ ]:
# Train baseline models
# models = {
#     'Linear Regression': LinearRegression(),
#     'Random Forest': RandomForestRegressor(n_estimators=100),
#     'Gradient Boosting': GradientBoostingRegressor()
# }
# 
# for name, model in models.items():
#     model.fit(X_train, y_train)
#     score = model.score(X_test, y_test)
#     print(f"{name} - R² Score: {score:.4f}")

In [ ]:
# Hyperparameter tuning
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [5, 10, 15],
#     'learning_rate': [0.01, 0.1, 0.5]
# }
# 
# gs = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5)
# gs.fit(X_train, y_train)
# print(f"Best parameters: {gs.best_params_}")
# print(f"Best score: {gs.best_score_:.4f}")

In [ ]:
# Save best model
# joblib.dump(gs.best_estimator_, '../models/best_model.pkl')
# print("Model saved to ../models/best_model.pkl")